In [ ]:
import aoc
import tqdm

In [ ]:
red_tiles = aoc.read("day9.txt").splitlines()
tile_locs = []
for t in red_tiles:
    p = t.split(',')
    tile_locs.append((int(p[0]), int(p[1])))

In [ ]:
max_area = 0
for i, t1 in enumerate(tile_locs):
    for t2 in tile_locs[i+1:]:
        area = abs(t1[0] - t2[0] + 1) * abs(t1[1] - t2[1] + 1)
        if area > max_area: 
            max_area = area

print(max_area)

# Part 2
Inspired by Day 10, 2023, I want to use the even-odd approach to find out what is inside and what is outside. From that, I can create ranges to find out which range in each row is "inside" (i.e. red or green tile). Then I can build the squares on a row by row basis, and check of the row of the square is completely encompassed by the inside range.

It's a bit complicated, but it is clear from the 100K x 100K grid that filling the grid and looping over it is not feasible/efficient. 

To use the even-odd approach, I have find all vertical walls (easy; just look whether a corner is directly above or beneath the following corner) and all the corners that change the inside: This is always the case if the wall comes from a vertical direction to change to a horizontal direction, but depends on the previous corner if it comes from a horizontal direction to switch to a vertical direction

```
   F-J  : F and J switch the even-odd 
F--J |  : F switches even and odd, J doesn't
L-7  |  : L switches even and odd, 7 doesn't
  L--J  : L and J switch the even and odd\
```
This now turns into:
```
   | |
|    | 
|    |
  |  | 
```

From here, I can write the indexes of each even/odd switcher, construct the inside ranges as intervals by popping the left two switchers as start and end of the interval per row, and the comparing each square per row.

In [ ]:
from collections import defaultdict, deque
from interval import Interval


def get_corner_type(prev, current, next):
    assert prev[0] != next[0] and prev[1] != next[1], f"Not a corner: {prev, current, next}"
    up = down = left = right = False
    if prev[0] < current[0]:
        left = True
    elif prev[0] > current[0]:
        right = True
    elif prev[1] < current[1]:
        up = True
    elif prev[1] > current[1]:
        down = True
    
    if next[0] > current[0]:
        if up:
            return "L"
        if down:
            return "F"
    elif next[0] < current[0]:
        if up:
            return "J"
        if down:
            return "7"
    elif next[1] > current[1]:
        if left:
            return "7"
        if right:
            return "F"
    elif next[1] < current[1]:
        if left:
            return "J"
        if right:
            return "L"

corners = tile_locs
corners_looped = [corners[-1]] + corners + [corners[0]]

corner_types = defaultdict(list)
for prev, current, next in zip(corners_looped[:-2], corners_looped[1:-1], corners_looped[2:]):
    current_corner_type = get_corner_type(prev, current, next)
    corner_types[current[1]].append((current[0], get_corner_type(prev, current, next)))

In [ ]:
inside_switchers = defaultdict(list)

# from corners
for r, l in corner_types.items():
    q = deque(sorted(l))
    last_ct = None
    while q:
        c, ct = q.popleft()
        if last_ct is None:
            assert ct in {"L", "F"}
        if ct in {"L", "F"}:
            inside_switchers[r].append(c)
        elif ct == "J" and last_ct == "L":
            inside_switchers[r].append(c)
        elif ct == "7" and last_ct == "F":
            inside_switchers[r].append(c)
        last_ct = ct

# Add normal vertical walls
x1, y1 = tile_locs[0]
for x2, y2 in tile_locs[1:] + [tile_locs[0]]:
    if x1 == x2:
        y_min, y_max = sorted([y1, y2])
        for y in range(y_min + 1, y_max):
            inside_switchers[y].append(x1)
    x1, y1 = x2, y2

In [ ]:
# Construct intervals with red and green tiles, per row
def merge_list_of_intervals(lst):
    intervals = deque(sorted(lst))
    combined_intervals = []
    current_interval = intervals.popleft()
    while intervals:
        i = intervals.popleft()
        try:
            current_interval = current_interval.merge(i)
        except ValueError:
            combined_intervals.append(current_interval)
            current_interval = i
    combined_intervals.append(current_interval)
    return combined_intervals

x_intervals_intermediate = defaultdict(list)
x_intervals = {}
for r, l in inside_switchers.items():
    q = deque(sorted(l))
    lo = None
    while q:
        lo = q.popleft()
        hi = q.popleft()
        x_intervals_intermediate[r].append(Interval(lo, hi))
    x_intervals[r] = merge_list_of_intervals(x_intervals_intermediate[r])

In [ ]:
def is_valid_square(t1, t2, x_intervals):
    x_min, x_max = sorted([t1[0], t2[0]])
    y_min, y_max = sorted([t1[1], t2[1]])
    base_interval = Interval(x_min, x_max)
    for r in range(y_min, y_max):
        for other_int in x_intervals[r]:
            if base_interval.is_dominated(other_int):
                break
            return False
    return True

max_area = 0
for i, t1 in tqdm.tqdm(enumerate(tile_locs)):
    for t2 in tile_locs[i+1:]:
        area = (abs(t1[0] - t2[0]) + 1) * (abs(t1[1] - t2[1]) + 1)
        if area > max_area: # This is less expensive than the validity check
            if is_valid_square(t1, t2, x_intervals):
                max_area = area
max_area